In [1]:
import requests 
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
from openai import OpenAI
import tiktoken

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [3]:
es_client = Elasticsearch('http://localhost:9200') 

In [59]:
client = OpenAI()

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-homework"

es_client.indices.create(index=index_name, body=index_settings)

In [6]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|████████████████████████████████████████████████████████████████████████| 948/948 [00:02<00:00, 340.82it/s]


In [7]:
query = 'How do I execute a command in a running docker container?'

In [8]:
def elastic_search(query):
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [86]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

    context_template = """
Q: {question}
A: {text}
""".strip()

    context = ''
    
    for doc in search_results:
        context += context_template.format(question=query, text=doc['text'])
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [81]:
search_results=elastic_search(query)


/tmp/ipykernel_49698/1747385194.py:22: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


In [97]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [98]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
rag(query)

In [100]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 62.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 796.9/796.9 kB 13.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [103]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [108]:
print(encoding.encode(prompt))

[63842, 261, 4165, 14029, 29186, 13, 30985, 290, 150339, 4122, 402, 290, 31810, 8099, 591, 290, 40251, 7862, 558, 8470, 1606, 290, 19719, 591, 290, 31810, 8099, 1261, 55959, 290, 150339, 364, 107036, 25, 3253, 621, 357, 15792, 261, 6348, 306, 261, 6788, 62275, 9282, 1715, 10637, 50738, 734, 48, 25, 3253, 621, 357, 15792, 261, 6348, 306, 261, 6788, 62275, 9282, 3901, 32, 25, 41281, 290, 9282, 3621, 306, 25383, 6766, 326, 151187, 290, 7251, 4859, 11, 813, 484, 480, 13217, 261, 38615, 6348, 558, 68923, 2461, 533, 278, 2230, 7962, 4859, 38615, 464, 3365, 523, 3335, 290, 9282, 382, 4279, 6788, 11, 15792, 261, 6348, 306, 290, 4857, 9282, 734, 68923, 10942, 350, 6555, 290, 9282, 26240, 446, 68923, 25398, 533, 278, 464, 6896, 26240, 29, 38615, 198, 6103, 277, 10732, 391, 79771, 8, 48, 25, 3253, 621, 357, 15792, 261, 6348, 306, 261, 6788, 62275, 9282, 3901, 32, 25, 1608, 665, 5150, 6291, 591, 634, 2698, 7342, 1511, 261, 91238, 9282, 2360, 290, 62275, 27776, 6348, 13, 44257, 1495, 316, 621, 480,